In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
from math import sqrt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [93]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
all_data = pd.concat((train_data.loc[:,'MSSubClass':'SaleCondition'],test_data.loc[:,'MSSubClass':'SaleCondition']))
matplotlib.rcParams['figure.figsize'] = (15.0, 6.0)
prices = pd.DataFrame({'price':train_data["SalePrice"],"log(price+1)":np.log1p(train_data["SalePrice"])})
#_ = prices.hist()

In [23]:
#x_train = train.iloc[:,:80]
#y_train = train.iloc[:,80:]
#x_train.shape
#scatter_matrix(x_train)
#x_train.hist(figsize=(18,14))
#plt.show()
#train_data.columns

In [128]:
###########################################################
# Remove columns that have low correlation
###########################################################
corr_matrix = train_data.corr()

# Uncomment for list of column correlation
#print(corr_matrix["SalePrice"].sort_values(ascending=False))

####TODO fix for negatively correlated
train_trimmed = train_data.copy()

pos_cols = corr_matrix["SalePrice"] > 0.1
#neg_cols = corr_matrix["SalePrice"] < -0.1 

train_trimmed = train_trimmed[pos_cols.index[pos_cols]]
#train_trimmed = train_trimmed[neg_cols.index[neg_cols]]

###########################################################
#Then remove all NaN entries, replace with mean
###########################################################
# Show NaN counts
#train_trimmed.isnull().sum()

# Based what the columns are, it may be better to drop those columns
train_trimmed.drop(['LotFrontage', 'GarageYrBlt'], axis=1, inplace=True)

# And replace with avg MasVnrArea because it only had 8 nan
train_trimmed = train_trimmed.fillna(train_trimmed.mean())

# Verify
#train_trimmed.isnull().sum()


############################################################
# Graveyard
#train_trimmed.drop([columns.index[columns]], axis=1, inplace=True) #| corr_matrix["SalePrice"] < 0.1
#train_trimmed.drop([train_trimmed[co]])

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [115]:
train_trimmed.shape

(1460, 23)

In [126]:
################################################################
# Shuffle our trimmed data and set aside test set
################################################################
def shuffleSplitData(data, train_size):
    #rand = np.random.randint()
    num_cols = data.shape[1]
    features = data.iloc[:,:num_cols-1]
    target = train_trimmed.iloc[:,num_cols-1:]
    X_train, X_test, y_train, y_test = train_test_split(
        data, target, train_size=train_size)
    return (X_train, X_test, y_train, y_test)

In [123]:
##################################################################
# Fit model
##################################################################
def trainModel(data, alpha, train_size, rmse=False):
    X_train, X_test, y_train, y_test = shuffleSplitData(data, train_size)
    clf = Ridge(alpha=alpha, solver='svd')
    if rmse == True:
        
    #_ = clf.fit(X_train, y_train)
    return

In [124]:
rmse= np.sqrt(-cross_val_score(clf, X_train, y_train, scoring="neg_mean_squared_error", cv = 5))